In [ ]:
from Extern import preproc_load_raw
from Extern import tmsi_poly5reader
from matplotlib import pyplot as plt
import scipy
from scipy.signal import spectrogram, hann, butter, filtfilt
from importlib import reload
import dat_preproc
import pandas as pd
import numpy as np
import os
import mne

In [ ]:
dat_path = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Extern_Trial\\572NR53_MedOn1_StimOn_L2b_FTGDir - 20230915T144713'

raw_file = os.path.join(
    dat_path,
    '572NR53_MedOn1_StimOn_L2b_FTGDir-20230915T144713.DATA.Poly5'
)

In [ ]:
raw = tmsi_poly5reader.Poly5Reader(raw_file)

In [ ]:
raw = raw.read_data_MNE()

In [ ]:
tstamps_sec = (1 / raw.info['sfreq']) * np.arange(raw.n_times)
new_ch_name = 'TimeinSec'
new_data_2d = tstamps_sec.reshape(1,-1)
new_info = mne.create_info(['Time_Sec'], raw.info['sfreq'], ch_types=['misc'])

# Create a RawArray object for the new channel
new_raw_array = mne.io.RawArray(new_data_2d, new_info)

# Add the new channel to the original Raw object
raw.add_channels([new_raw_array])

In [ ]:
fif_name = os.path.join(dat_path, 'Sub001_Extern_MedOn_StimOn_L2b.fif')
raw.save(fif_name, overwrite = True)

In [ ]:
raw = mne.io.read_raw_fif(os.path.join(
    'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\ENTRAINMENT\\Extern_Trial',
        'Sub001_Extern_MedOn_StimOn_L2.fif'
    )
)

In [ ]:
raw.ch_names

In [ ]:
reload(dat_preproc)
dat = raw.get_data(picks = 'LFPL01STN')[0]
detrended_signal = signal.detrend(dat)
filter_order = 5
b, a = scipy.signal.butter(filter_order, [2, 512], 
                btype='bandpass', output='ba', fs=raw.info['sfreq'])              
pass_filtered_dat = scipy.signal.filtfilt(b, a, detrended_signal) 


In [ ]:
%matplotlib qt
plt.plot(raw['Time_Sec'][1], pass_filtered_dat, label = '')
#plt.plot(raw['Time_Sec'][1], outputSignal)

In [ ]:


# Calculate the notch frequencies for the harmonics
harmonics = [130, 260, 390, 520, 650, 780, 910, 1040, 1170, 1300, 1430, 1560, 1690, 1820, 1950]  # You can adjust the number of harmonics as needed

samp_freq  = raw.info['sfreq']  # Sample frequency (Hz)
notch_freq  = 130
quality_factor  = 30.0  # Quality factor
# Design notch filter
b_notch, a_notch = signal.iirnotch(notch_freq, quality_factor, samp_freq)


In [ ]:
for harm in harmonics:
    b_notch, a_notch = signal.iirnotch(harm, quality_factor, samp_freq)
    outputSignal = signal.filtfilt(b_notch, a_notch, pass_filtered_dat)

In [ ]:
reload(dat_preproc)
%matplotlib qt
x = outputSignal
raw = raw
win_samp = 4096
noverlap = 0.25
window = hann(win_samp, sym=False)
fs = raw.info['sfreq']
f, t, Sxx = scipy.signal.spectrogram(x = x, fs = fs, window = window, noverlap = noverlap)

In [ ]:
plt.specgram(x = x, Fs = fs, noverlap = noverlap, cmap = 'viridis')
plt.ylim(2,512)

In [ ]:
harmonics = [130, 260, 390, 520, 650, 780, 910, 1040, 1170, 1300, 1430, 1560, 1690, 1820, 1950]  # You can adjust the number of harmonics as needed
len(harmonics)

In [ ]:
# Filter the list to select only elements containing 'LFPL'
#chans = [s for s in raw.ch_names if 'EEG' in s]
chans = ['LFPL01STN', 'LFPL02STN', 'LFPL03STN', 'LFPL04STN', 'LFPL08STN']
print(chans)

In [ ]:
%matplotlib qt
from scipy import signal
DETREND = 1

for ch in chans:
    dat = raw.get_data(picks = ch)[0]
    fs = raw.info['sfreq']
    stimulation_frequencies = [260, 390]
    Q = 30  # Q factor for notch filter
    window_length = int(fs)  # 1-second window length
    overlap = window_length // 4  # 25% overlap

    # Detrend the LFP signal
    if DETREND == 1:
        dat = signal.detrend(dat)

    # Calculate the notch frequencies for the stimulation frequency and its harmonics
    #harmonics = np.arange(1, 16)  # You can adjust the number of harmonics as needed
    #notch_frequencies = 130 * harmonics

    # Create an empty array to store filtered signals
    filtered_signals = []

    # Apply notch filters for each frequency and its harmonics
    for freq in stimulation_frequencies:
        w0 = freq / (fs / 2)  # Normalized notch frequency
        b, a = signal.iirnotch(w0, Q)
        filtered_signal = signal.filtfilt(b, a, dat)
        filtered_signals.append(filtered_signal)

    # Combine the filtered signals
    combined_signal = np.sum(filtered_signals, axis=0)

    # Calculate the short-time Fourier transform (STFT) using Hamming window
    frequencies, times, Zxx = signal.stft(combined_signal, fs=fs, nperseg=window_length, noverlap=overlap, window='hamming')

    # Plot the spectral amplitudes
    plt.figure()
    plt.pcolormesh(times, frequencies, 20 * np.log10(np.abs(Zxx)), shading='auto', cmap='inferno')
    plt.yticks(np.arange(0, 512, 30))
    plt.title(str(LSTN_chan))
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.colorbar(label='Amplitude (dB)')
    plt.ylim(1, 512)  # Set the frequency range as desired
    plt.show()

    #plt.savefig(os.path.join(dat_path, f'{LSTN_chan}_L2b_FFT'), dpi = 200)
